In [1]:
import pandas as pd
import datetime as dt
import json
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

In [2]:
# Read Train data with limitation
train_path = 'train.csv'
train_df = pd.read_csv(train_path)
#train_df = pd.read_csv(train_path, nrows = 5000, parse_dates=["pickup_datetime"])

In [3]:
train_df.head(5)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1


## Preprocessing data

#### Cleaning of illogical data

##### Removing Data with NA values

In [4]:
# Remove Training data with NA values
processed_train_df = train_df.dropna(inplace = False)

##### Removing Data with FARE < 0

In [5]:
# Removing fare amounts that are less than 0
processed_train_df = processed_train_df[processed_train_df["fare_amount"] > 0 ]

##### Removing Data with Longitude AND Latitude == 0 

In [6]:
# Remove Training data with "0" values for longitude, Latittube for pickup and drop off
processed_train_df = processed_train_df[(processed_train_df["pickup_longitude"] != 0) & (processed_train_df["pickup_latitude"] != 0) & (processed_train_df["dropoff_longitude"] != 0) & (processed_train_df["dropoff_latitude"] != 0) ]

##### Removing PASSENGER_COUNT < 1 and > 10

In [7]:
# Removing passenger counts that are more than 
processed_train_df = processed_train_df[(processed_train_df["passenger_count"] > 0) & (processed_train_df["passenger_count"] < 10) ]

In [8]:
processed_train_df.dtypes

key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
dtype: object

In [9]:
processed_train_df.isnull().values.any()

False

In [10]:
print("Number of rows dropped : ", train_df.shape[0] - processed_train_df.shape[0])
print("Number of rows remainingg: ", processed_train_df.shape[0] )

Number of rows dropped :  1299031
Number of rows remainingg:  54124825


In [11]:
processed_train_df.head(20)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
5,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.731630,-73.972892,40.758233,1
6,2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00 UTC,-73.980002,40.751662,-73.973802,40.764842,1
7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1
8,2012-12-03 13:10:00.000000125,9.0,2012-12-03 13:10:00 UTC,-74.006462,40.726713,-73.993078,40.731628,1
9,2009-09-02 01:11:00.00000083,8.9,2009-09-02 01:11:00 UTC,-73.980658,40.733873,-73.991540,40.758138,2


In [12]:
datetime_list = []
for datetime_str in processed_train_df['pickup_datetime']:
    datetime_value = dt.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S UTC')
    date = datetime_value.date() 
    time = datetime_value.time()
    day = datetime_value.weekday()
    weekend = (day>=4)
    datetime_list.append([date,time,date,weekend])
    #print(date)

In [13]:
date_series = pd.Series(v[0] for v in datetime_list)

In [14]:
time_series = pd.Series(v[1] for v in datetime_list)

In [15]:
day_series = pd.Series(v[2] for v in datetime_list)

In [16]:
weekend_series = pd.Series(v[3] for v in datetime_list)

In [17]:
cal = calendar()
holidays = cal.holidays(start= date_series.min(), end=date_series.max())
holidays = [x.date() for x in holidays]
print("Earliest Date in Train Datatset:",date_series.min())
print("Latest Date in Train Dataset:",date_series.max())

Earliest Date in Train Datatset: 2009-01-01
Latest Date in Train Dataset: 2015-06-30


In [18]:
holiday_list = []
for date in date_series: 
    if date in holidays:
        holiday_list.append(int(1))
    else: 
        holiday_list.append(int(0))   
holiday_series = pd.Series(v for v in holiday_list)

In [19]:
holiday_series.value_counts()

0    52992856
1     1131969
dtype: int64

#### Checking if the column size

In [20]:
date_series.count()

54124825

In [21]:
holiday_series.count()

54124825

In [22]:
time_series.count()

54124825

In [23]:
day_series.count()

54124825

In [24]:
weekend_series.count()

54124825

In [25]:
new_combined = processed_train_df.copy()
new_combined = new_combined.reset_index() 

In [26]:
new_combined['date'] = date_series
new_combined['time'] = time_series
new_combined['day'] = day_series
new_combined['weekend'] = weekend_series
new_combined['holiday'] = holiday_series

In [27]:
new_combined.head(25)

,index,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,date,time,day,weekend,holiday
0,0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15,17:26:21,2009-06-15,False,0
1,1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05,16:52:16,2010-01-05,False,0
2,2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-18,00:35:00,2011-08-18,False,0
3,3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21,04:30:42,2012-04-21,True,0
4,4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09,07:51:00,2010-03-09,False,0
5,5,2011-01-06 09:50:45.0000002,12.1,2011-01-06 09:50:45 UTC,-74.000964,40.731630,-73.972892,40.758233,1,2011-01-06,09:50:45,2011-01-06,False,0
6,6,2012-11-20 20:35:00.0000001,7.5,2012-11-20 20:35:00 UTC,-73.980002,40.751662,-73.973802,40.764842,1,2012-11-20,20:35:00,2012-11-20,False,0
7,7,2012-01-04 17:22:00.00000081,16.5,2012-01-04 17:22:00 UTC,-73.951300,40.774138,-73.990095,40.751048,1,2012-01-04,17:22:00,2012-01-04,False,0
8,8,2012-12-03 13:10:00.000000125,9.0,2012-12-03 13:10:00 UTC,-74.006462,40.726713,-73.993078,40.731628,1,2012-12-03,13:10:00,2012-12-03,False,0
9,9,2009-09-02 01:11:00.00000083,8.9,2009-09-02 01:11:00 UTC,-73.980658,40.733873,-73.991540,40.758138,2,2009-09-02,01:11:00,2009-09-02,False,0


In [28]:
new_combined.dtypes

index                  int64
key                   object
fare_amount          float64
pickup_datetime       object
pickup_longitude     float64
pickup_latitude      float64
dropoff_longitude    float64
dropoff_latitude     float64
passenger_count        int64
date                  object
time                  object
day                   object
weekend                 bool
holiday                int64
dtype: object

In [29]:
new_combined.isnull().values.any()
# new_combined['holiday']=new_combined['holiday'].astype('int') 
# new_combined['weekend']=new_combined['weekend'].astype('int') 


False

In [30]:
new_combined.shape

(54124825, 14)

In [31]:
new_combined

,index,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,date,time,day,weekend,holiday
0,0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,2009-06-15,17:26:21,2009-06-15,False,0
1,1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,2010-01-05,16:52:16,2010-01-05,False,0
2,2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,2011-08-18,00:35:00,2011-08-18,False,0
3,3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2012-04-21,04:30:42,2012-04-21,True,0
4,4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2010-03-09,07:51:00,2010-03-09,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54124820,55423851,2014-03-15 03:28:00.00000070,14.0,2014-03-15 03:28:00 UTC,-74.005272,40.740027,-73.963280,40.762555,1,2014-03-15,03:28:00,2014-03-15,True,0
54124821,55423852,2009-03-24 20:46:20.0000002,4.2,2009-03-24 20:46:20 UTC,-73.957784,40.765530,-73.951640,40.773959,1,2009-03-24,20:46:20,2009-03-24,False,0
54124822,55423853,2011-04-02 22:04:24.0000004,14.1,2011-04-02 22:04:24 UTC,-73.970505,40.752325,-73.960537,40.797342,1,2011-04-02,22:04:24,2011-04-02,True,0
54124823,55423854,2011-10-26 05:57:51.0000002,28.9,2011-10-26 05:57:51 UTC,-73.980901,40.764629,-73.870605,40.773963,1,2011-10-26,05:57:51,2011-10-26,False,0


In [ ]:
new_combined.to_csv('c.csv', index=False)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Jeng Wen\Anaconda3\envs\ml\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\JENGWE~1\AppData\Local\Temp/ipykernel_1816/2576653593.py", line 1, in <module>
    new_combined.to_csv('checkpoint-1.csv', index=False)
NameError: name 'new_combined' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jeng Wen\Anaconda3\envs\ml\lib\site-packages\IPython\core\interactiveshell.py", line 2064, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jeng Wen\Anaconda3\envs\ml\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(

TypeError: object of type 'NoneType' has no len()